# Домашнє завдання: Внесення оновлень в БД і робота з транзакціями

Це ДЗ передбачене під виконання на локальній машині. Виконання з Google Colab буде суттєво ускладнене.

## Підготовка
1. Переконайтесь, що у вас встановлены необхідні бібліотеки:
   ```bash
   pip install sqlalchemy pymysql pandas matplotlib seaborn python-dotenv
   ```

2. Створіть файл `.env` з параметрами підключення до бази даних classicmodels. Базу даних ви можете отримати через

  - docker-контейнер згідно існтрукції в [документі](https://www.notion.so/hannapylieva/Docker-1eb94835849480c9b2e7f5dc22ee4df9), також відео інструкції присутні на платформі - уроки "MySQL бази, клієнт для роботи з БД, Docker і ChatGPT для запитів" та "Як встановити Docker для роботи з базами даних без терміналу"
  - або встановивши локально цю БД - для цього перегляньте урок "Опціонально. Встановлення MySQL та  БД Сlassicmodels локально".
  
  Приклад `.env` файлу ми створювали в лекції. Ось його обовʼязкове наповнення:
    ```
    DB_HOST=your_host
    DB_PORT=3306 або 3307 - той, який Ви налаштували
    DB_USER=your_username
    DB_PASSWORD=your_password
    DB_NAME=classicmodels
    ```
  Якщо ви створили цей файл під час перегляду лекції - **новий створювати не треба**. Замініть лише назву БД, або пропишіть назву в коді створення підключення (замість отримання назви цільової БД зі змінних оточення). Але переконайтесь, що до `.env` файл лежить в тій самій папці, що і цей ноутбук.

  **УВАГА!** НЕ копіюйте скрит для **створення** `.env` файлу. В лекції він наводиться для прикладу. І давалось пояснення, що в реальних проєктах ми НІКОЛИ не пишемо доступи до бази в коді. Копіювання скрипта для створення `.env` файлу сюди в ДЗ буде вважатись грубою помилкою і ми зніматимемо бали.

3. Налаштуйте підключення через SQLAlchemy до БД за прикладом в лекції.

Рекомендую вивести (відобразити) змінну engine після створення. Вона має бути не None! Якщо None - значить у Вас не підтягнулись налаштування з .env файла.

Ви також можете налаштувати параметри підключення до БД без .env файла, просто прописавши текстом в відповідних місцях. Це - не рекомендований підхід.


## Завдання

### Завдання 1: Оновлення інформації про клієнта (2 бали)

**Створіть функцію для оновлення контактної інформації клієнта** з наступними можливостями:
- Оновлення телефону клієнта
- Оновлення email (якщо поле існує)
- Логування змін в окрему таблицю

Використайте підхід з параметризованими запитами через `text()` та `UPDATE` оператор.

Запустіть функцію і продемонструйте її роботу, запустивши SELECT, який допоможе це зробити.



In [1]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

In [4]:
# Підключення до БД
def create_connection():
    load_dotenv()

    host = os.getenv('DB_HOST')
    port = os.getenv('DB_PORT', '3306')
    user = os.getenv('DB_USER')
    password = os.getenv('DB_PASSWORD')
    database = os.getenv('DB_NAME')

    connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"

    engine = create_engine(
        connection_string,
        pool_size=1,
        max_overflow=20,
        pool_pre_ping=True,
        echo=False
    )

    return engine

engine = create_connection()

In [17]:
with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS customer_update_log (
            log_id INT AUTO_INCREMENT PRIMARY KEY,
            customerNumber INT,
            field_updated VARCHAR(50),
            old_value VARCHAR(255),
            new_value VARCHAR(255),
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
    """))
    conn.commit()


In [20]:
def update_customer_contact(engine, cust_num, new_phone):
    with engine.connect() as conn:
        # Отримати старе значення телефону
        old_phone_query = text("""
            SELECT phone FROM customers WHERE customerNumber = :cust_num
        """)
        result = conn.execute(old_phone_query, {"cust_num": cust_num})
        row = result.fetchone()
        if row is None:
            print("Клієнта не знайдено.")
            return

        old_phone = row[0]  # <--- ось тут зміна

        # Оновити телефон
        update_query = text("""
            UPDATE customers
            SET phone = :new_phone
            WHERE customerNumber = :cust_num
        """)
        conn.execute(update_query, {"new_phone": new_phone, "cust_num": cust_num})

        # Записати лог
        log_query = text("""
            INSERT INTO customer_update_log (customerNumber, field_updated, old_value, new_value)
            VALUES (:cust_num, 'phone', :old_val, :new_val)
        """)
        conn.execute(log_query, {
            "cust_num": cust_num,
            "old_val": old_phone,
            "new_val": new_phone
        })

        conn.commit()
        print(f"Телефон клієнта {cust_num} оновлено з {old_phone} на {new_phone}")


In [21]:
update_customer_contact(engine, 103, "+380990001122")


Телефон клієнта 103 оновлено з 888-777-9999 на +380990001122


In [22]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM customer_update_log ORDER BY updated_at DESC LIMIT 5"))
    df_log = pd.DataFrame(result.fetchall(), columns=result.keys())
    print(df_log)


   log_id  customerNumber field_updated     old_value      new_value  \
0       1             103         phone  888-777-9999  +380990001122   

           updated_at  
0 2025-08-01 08:04:56  


### Завдання 2: Створення нового замовлення з транзакцією (5 балів)

**Реалізуйте процес створення нового замовлення** з наступними кроками в одній транзакції:
- Створення запису в таблиці `orders`
- Додавання товарних позицій в `orderdetails`
- Перевірка наявності товарів на складі
- Зменшення кількості товарів на складі

Запустіть процес з тестовими даними і продемонструйте через SELECT, що процес успішно відпрацював і були виконані необхідні операції.




In [25]:
from datetime import datetime
from sqlalchemy.exc import SQLAlchemyError

# Тестові дані
customer_number = 103
new_order_number = 99999
order_date = datetime.now().date()
required_date = datetime(2025, 8, 10).date()
shipped_date = None
status = 'In Process'
comments = 'Test order from script'

# Список товарів: productCode, quantityOrdered, priceEach
order_items = [
    {'productCode': 'S10_1678', 'quantityOrdered': 2, 'priceEach': 95.70},
    {'productCode': 'S10_1949', 'quantityOrdered': 1, 'priceEach': 53.80},
]

with engine.connect() as conn:
    trans = conn.begin()
    try:
        # 1. Створення замовлення
        conn.execute(text("""
            INSERT INTO orders (orderNumber, orderDate, requiredDate, status, comments, customerNumber)
            VALUES (:orderNumber, :orderDate, :requiredDate, :status, :comments, :customerNumber)
        """), {
            'orderNumber': new_order_number,
            'orderDate': order_date,
            'requiredDate': required_date,
            'status': status,
            'comments': comments,
            'customerNumber': customer_number
        })

        # 2. Додавання до orderdetails та 3-4. Перевірка і оновлення складу
        for idx, item in enumerate(order_items, start=1):
            # Перевірка наявності товару
            stock_result = conn.execute(text("""
                SELECT quantityInStock FROM products WHERE productCode = :productCode
            """), {'productCode': item['productCode']}).scalar_one()

            if stock_result < item['quantityOrdered']:
                raise ValueError(f"Insufficient stock for product {item['productCode']}")

            # Додавання в orderdetails
            conn.execute(text("""
                INSERT INTO orderdetails (orderNumber, productCode, quantityOrdered, priceEach, orderLineNumber)
                VALUES (:orderNumber, :productCode, :quantityOrdered, :priceEach, :orderLineNumber)
            """), {
                'orderNumber': new_order_number,
                'productCode': item['productCode'],
                'quantityOrdered': item['quantityOrdered'],
                'priceEach': item['priceEach'],
                'orderLineNumber': idx
            })

            # Зменшення товару на складі
            conn.execute(text("""
                UPDATE products
                SET quantityInStock = quantityInStock - :qty
                WHERE productCode = :productCode
            """), {
                'qty': item['quantityOrdered'],
                'productCode': item['productCode']
            })

        trans.commit()
        print("✅ Замовлення створено успішно!")

    except (SQLAlchemyError, ValueError) as e:
        trans.rollback()
        print("❌ Помилка при створенні замовлення:", e)


✅ Замовлення створено успішно!


In [26]:
# Перевірка замовлення
with engine.connect() as conn:
    order = conn.execute(text("SELECT * FROM orders WHERE orderNumber = :orderNumber"), {
        'orderNumber': new_order_number
    }).fetchone()

    details = conn.execute(text("SELECT * FROM orderdetails WHERE orderNumber = :orderNumber"), {
        'orderNumber': new_order_number
    }).fetchall()

    print("🔍 Order:")
    print(order)

    print("\n🧾 Order Details:")
    for row in details:
        print(row)


🔍 Order:
(99999, datetime.date(2025, 8, 1), datetime.date(2025, 8, 10), None, 'In Process', 'Test order from script', 103)

🧾 Order Details:
(99999, 'S10_1678', 2, Decimal('95.70'), 1)
(99999, 'S10_1949', 1, Decimal('53.80'), 2)
